In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/big-mart-salescsv/Train_UWu5bXk.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

two coins need cleaning

In [ ]:
data.info()

In [ ]:
data.plot.scatter(x='Item_Visibility',
                 y='Item_Outlet_Sales',
                 title='Visibility vs sales')

In [ ]:
data.corr()

### Insight:
visibility of less than 0.2 seem to have more sales

### Outlet vs Sales

In [ ]:
data.groupby('Outlet_Identifier').Item_Outlet_Sales.sum().sort_values().plot.barh(title='Outlet vs sales')

### Insight:
- Outlet 27 is hightest sales
- Outlet 10 and 19 are the lowest

### Item type vs Sales

In [ ]:
data.groupby('Item_Type').Item_Outlet_Sales.sum().sort_values().plot.barh(title='Item Type vs sales')

### Insight:
- Fruit and vege and Snack are among the top selling items
- Breakfast and Seafood are not selling much

Feature engineering Ideas:
- Create a lebelled Visibility - low, high
- Reduce Item_Type categories to food, drink, non-consuable
- Operational year = 2013 - est Year

## Data Pre-Processing

### Item_Fat_Content

In [ ]:
data.Item_Fat_Content.value_counts()

In [ ]:
data.Item_Fat_Content.replace(to_replace=['LF','low fat'], value='Low Fat', inplace=True)
data.Item_Fat_Content.replace(to_replace=['reg'], value='Regular', inplace=True)
data.Item_Fat_Content.value_counts()

Item_Fat_Content is correctly labelled

In [ ]:
data.isnull().sum()

### Item_Weight

In [ ]:
data.Item_Identifier.value_counts()

In [ ]:
data.Item_Weight = data.Item_Weight.fillna(data.groupby('Item_Identifier').Item_Weight.transform('mean'))
data.isnull().sum()

In [ ]:
data[data.Item_Weight.isnull()]

In [ ]:
data.Item_Weight = data.Item_Weight.fillna(data.groupby('Item_Type').Item_Weight.transform('mean'))
data.isnull().sum()

Item_Weight is imputed with groups formed by Item_Identifier and Item_Type

Outlet_Size

In [ ]:
data.Outlet_Size.value_counts(dropna=False)

In [ ]:
pd.crosstab(index=data.Outlet_Size, columns= data.Outlet_Type)

all grocery stores are small

In [ ]:
data.Outlet_Size = data.Outlet_Size.fillna(data.Outlet_Type.map({'Grocery Store': 'Small'}))
data.Outlet_Size.value_counts(dropna=False)

In [ ]:
pd.crosstab(index= data.Outlet_Size , columns=  data.Outlet_Location_Type)

In [ ]:
data.Outlet_Size = data.Outlet_Size.fillna(data.Outlet_Location_Type.map({'Tier 2': 'Small'}))
data.Outlet_Size.value_counts(dropna=False)

Outlet_Size imputed using Outlet_Type and Outlet_Location_Type

In [ ]:
data.isnull().sum()

In [ ]:
Convert to numerical

In [ ]:
data.select_dtypes(include=[object]).columns.to_list()

In [ ]:
drop_cols = ['Item_Identifier', 'Outlet_Identifier', 'Item_Outlet_Sales']

cat_cols = ['Item_Identifier',
 'Item_Fat_Content',
 'Item_Type',
 'Outlet_Identifier',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
for col in cat_cols:
    data[col] = le.fit_transform(data[col])

    
data.head()

## Model Building / Train

In [ ]:
from sklearn.model_selection import train_test_split
x = data.drop(columns=drop_cols)
y = data.Item_Outlet_Sales

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error

In [ ]:
linear_model1 = LinearRegression()
linear_model1.fit(x_train, y_train)
y_pred = linear_model1.predict(x_test)


In [ ]:
accuracy = linear_model1.score(x_test, y_test)
accuracy 

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
rmse